In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import date

Data Loading

In [2]:

def load_data(file):
    mdro = pd.read_excel(file, sheet_name='MDROs', 
                        dtype={'病歷號碼':'str', '申請編號':'int', '就醫序號':'int'})
    report = pd.read_excel(file, sheet_name='微生物報告', 
                        dtype={'病歷號碼':'str', '申請編號':'int', '就醫序號':'int'})
    info = pd.read_excel(file, sheet_name='就醫資訊', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    # anti = pd.read_excel(file, sheet_name='抗生素使用', 
    #                     dtype={'病歷號碼':'str', '就醫序號':'int'})
    examValue = pd.read_excel(file, sheet_name='檢驗數值', 
                        dtype={'病歷號碼':'str', '就醫序號':'int'})
    
    # data processing 
    mdro = mdro[['病歷號碼', 'time', '就醫序號','申請編號', '菌株', '預測值', '部位', 'sendTime']]
    mdro.rename({'time':'openTime'}, axis=1, inplace=True)
    
    report = report[['病歷號碼', 'bed', '就醫序號','申請編號', 'strain','部位', '抗藥性菌株', 'examTime','reportTime']]
    
    cdss = pd.merge(report, mdro, how='left', on=['病歷號碼', '就醫序號','申請編號', '部位'])
    cdss = cdss.drop_duplicates()
    cdss['miniTime_Open'] = cdss.groupby(['病歷號碼', 'bed','就醫序號', '申請編號', '部位', '菌株', '預測值',  'sendTime'])['openTime'].transform('min')
    
    info = info[['病歷號碼', '就醫序號', '性別', '生日', '年齡', '來院日期', '離院日期', '天數', '門急住', '科別代碼', '科別']]
    info.rename({'門急住':'bed'}, axis=1, inplace=True)
    
    cdss_info = pd.merge(cdss, info, how='left', on=['病歷號碼', '就醫序號', 'bed'])
    
    # Filtering mini open CDSS time
    CDSS = cdss_info[cdss_info.openTime==cdss_info.miniTime_Open].drop_duplicates()
    
    # Add column : max time leaving hospital
    CDSS.離院日期 = pd.to_datetime(CDSS.離院日期).dt.date
    CDSS['MaxTime_leave'] = CDSS.groupby(['病歷號碼', 'bed','就醫序號', '申請編號', '部位', '菌株', '預測值', 'sendTime', '來院日期'])['離院日期'].transform('max')
    # CDSS = pd.merge(cdss_info, anti, how='left', on=['病歷號碼','就醫序號'])
    # CDSS = CDSS.drop_duplicates()
    
    # Predict Level
    condition_1 = [(CDSS['預測值'] <= 20), (CDSS['預測值'] > 20)&(CDSS['預測值'] <= 40), 
             (CDSS['預測值'] > 40)&(CDSS['預測值'] <= 60), (CDSS['預測值'] > 60)&(CDSS['預測值'] <= 80),
             (CDSS['預測值'] > 80)&(CDSS['預測值'] <= 100)]

    value_1 = ['0~20%', '21~40%', '41~60%', '61~80%', '81~100%']
    CDSS['PredLevel'] = np.select(condition_1, value_1) 
    
    # Stain Category
    condition_2 =[
                  (CDSS['菌株'].isin(['Enterococcus faecium(VRE)', 'Enterococcus faecium', 'Enterococcus faecalis'])),
                  (CDSS['菌株'].isin(['Morganella morganii'])),
                  (CDSS['菌株'].isin(['Klebsiella pneumoniae'])),
                  (CDSS['菌株'].isin(['Pseudomonas aeruginosa'])),
                  (CDSS['菌株'].isin(['Escherichia coli'])),
                  (CDSS['菌株'].isin(['Acinetobacter baumannii', 'Acinetobacter baumannii/calcoaceticus complex'])),
                  (CDSS['菌株'].isin(['Staphylococcus aureus', 'Staphylococcus aureus(MRSA)'])),
                  (CDSS['菌株'].isin(['Enterobacter cloacae']))
                ]
    
    value_2 = ['Enterococcus faecium', 'Morganella morganii', 'Klebsiella pneumoniae', 'Pseudomonas aeruginosa',
               'Escherichia coli', 'Acinetobacter baumannii', 'Staphylococcus aureus', 'Enterobacter cloacae']
    
    CDSS['Group_Strain'] = np.select(condition_2, value_2) 
    
    # exam Year & Month
    CDSS["examDate"] = pd.to_datetime( 
                                  pd.to_datetime(CDSS.examTime).dt.year.map(str) + '-' + \
                                  pd.to_datetime(CDSS.examTime).dt.month.map(str) + '-' + \
                                  pd.to_datetime(CDSS.examTime).dt.day.map(str)
                                )
    
    CDSS['Resist_bool'] = np.where(CDSS['抗藥性菌株']==1, True, False)
    CDSS.rename({'年齡':'Age', '預測值':'PredValue'}, axis=1, inplace=True)
    
    return CDSS
    

In [3]:
# 智抗菌平台報表_20220331.xlsx

# file = r'C:\Users\User\VSCode\智抗菌平台報表_20220331.xlsx'
file = r'C:\Users\User\VSCode\CDSS\output\智抗菌平台報表_20220502.xlsx'
CDSS = load_data(file)
# CDSS.head(10)

In [4]:
old_file = r'C:\Users\User\VSCode\CDSS\source\智抗菌平台報表_20220309.xlsx'
old_file_1 = r'C:\Users\User\VSCode\CDSS\source\智抗菌平台報表_20220331.xlsx'
CDSS_old = load_data(old_file)
CDSS_old_1 = load_data(old_file_1)
CDSS = pd.concat([CDSS_old, CDSS_old_1, CDSS])

# Filter Fault Time
CDSS['sendTime'] = pd.to_datetime(CDSS['sendTime'])
CDSS = CDSS[(CDSS['sendTime'] < pd.to_datetime('2022-03-07 00:00')) | (CDSS['sendTime'] > pd.to_datetime('2022-03-21 23:59'))]

CDSS = CDSS.drop_duplicates().reset_index(drop=True).sort_values(['病歷號碼', '申請編號', 'examTime'])

In [5]:
# Output File
# CDSS.to_excel(r'C:\Users\User\VSCode\CDSS_report_mdro_info.xlsx')
# CDSS.to_excel(r'C:\Users\User\VSCode\CDSS\output\CDSS_0502.xlsx')